In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException, NoSuchElementException, TimeoutException, StaleElementReferenceException, UnexpectedAlertPresentException
from selenium.webdriver.common.by import By
import os
import urllib.request
from collections import Counter
import time
import pickle


class Insta():
    
    user_ids = []
    dataset = [['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url']]
    keyword_dic = {}
    
    def __init__(self, keyword, number, file_name='images', mode='None'):
        self.keyword = keyword
        self.number = number
        self.file_name = file_name
        self.mode = mode

    def chrome_open(self):

        self.options = webdriver.ChromeOptions()
        if self.mode == 'headless':
            self.options.add_argument('headless')
        self.options.add_argument('window-size=1920x1080')
        self.options.add_argument("disable-gpu")
        self.options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        self.options.add_argument("lang=ko_KR") # 한국어!
        self.options.add_argument("disable-infobars")
        self.driver = webdriver.Chrome('chromedriver', chrome_options=self.options)

        self.driver.get('http://google.com')
        self.driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")
        self.driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
        self.driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")

        self.driver.implicitly_wait(3)
        
    def insta_keyword(self):
        keyword_t = urllib.parse.urlencode({'': self.keyword})[1:]
        self.driver.get('https://www.instagram.com/explore/tags/{}/?hl=ko'.format(keyword_t))
        #iframe = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'fb_xdm_frame_https')))
        #self.driver.switch_to.frame(iframe1)
        self.total = int(self.driver.find_element_by_xpath(
            '//*[@id="react-root"]/section/main/article/header/div[2]/span/span').text.replace(',', ''))

        
    def crawling(self):
        
        if self.number == 'total':
            number = self.total
        else:
            number = self.number
               
        enter_post = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.eLAPa')))        
        enter_post.click()
        main_text = None
        check_text = True
        checker2 = 0
        for i in range(int(number)):
            if i != 0: check_text = main_text
            start_checker = 0
            same_checker = 0
            #print(start_checker, same_checker, 1)
            #print(start_checker, same_checker, 2)
            while True:       
                try:
                    texts = WebDriverWait(self.driver, 3).until(
                            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'ul.Xl2Pu > li')))                
                    main_text = texts[0].text
                    comments = [c.text for c in texts[1:]]
                    if check_text == main_text:
                        same_checker += 1
                        if same_checker - start_checker > 200:
                            print(start_checker, same_checker, 3)
                            break
                        continue
                    else: 
                        user_id = WebDriverWait(self.driver, 3).until(
                            EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.e1e1d'))).text
                        print('get user_id\n\n')
                        break
                except NoSuchElementException as ex:
                    print(i, 'user_id', ex)
                    same_checker += 1
                    if same_checker - start_checker > 400:
                        print(start_checker, same_checker, 3)
                        break
                except Exception as e:
                    same_checker += 1
                    print(i, 'user_id', e)
                    if same_checker - start_checker > 400:
                        print(start_checker, same_checker, 3)                    
                        break
                        
            if same_checker - start_checker > 200:
                try:
                    next_button = self.driver.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow')
                    next_button.click()
                #print(start_checker, same_checker, 4)
                except NoSuchElementException as ex:
                    checker2 += 1
                    print(i, 'user_id', ex)
                    if checker2 > 5:
                        break
                except Exception as e:
                    checker2 += 1
                    print(i, 'user_id', e)
                    if checker2 > 50:
                        break
                continue
            '''        
            if '흡기' in main_text:
                next_button = self.driver.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow')
                next_button.click()
                continue
            '''
            

            if not user_id in self.user_ids:
                self.user_ids.append(user_id)
                self.keyword_dic[user_id] = {'user_index' : i, 'user_id' : user_id, 'main_text' : [], 'hashtags' : [], 'created_at': [],'comments' : [], 'likes' : [], 'current_url' : []}
                while True:
                    try:
                        hashtags = [h.text[1:] for h in self.driver.find_elements_by_css_selector('ul.Xl2Pu>li>span>a')]
                        #print(1)
                        created_at = self.driver.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')
                        #print(2)
                        current_url = self.driver.current_url
                        #print(3)
                        
                        # comments = [c.text for c in texts[1:]]
                        # likes = article.find_element_by_css_selector('span.zV_Nj>span').text

                    except NoSuchElementException as ex:
                        print(i, 1, user_id, ex)
                    except Exception as e:
                        print(i, 1, user_id, e)
                        
                    else:
                        self.keyword_dic[user_id]['main_text'].append(main_text)
                        self.keyword_dic[user_id]['hashtags'].append(hashtags)
                        self.keyword_dic[user_id]['created_at'].append(created_at)
                        self.keyword_dic[user_id]['current_url'].append(current_url)
                        self.keyword_dic[user_id]['comments'].append('/'.join(comments))
                        likes = ''
                        try:
                            likes = self.driver.find_element_by_css_selector('span.zV_Nj>span').text
                            self.keyword_dic[user_id]['likes'].append(likes)
                            self.dataset.append([user_id, created_at, main_text, hashtags, comments, likes, current_url])
                            break

                        except:
                            print('below 11')
                            self.keyword_dic[user_id]['likes'].append(likes)
                            self.dataset.append([user_id, created_at, main_text, hashtags, comments, likes, current_url])
                            break
                        # self.keyword_dic[user_id]['comments'].append(comments)
                        # self.keyword_dic[user_id]['likes'].append(likes)
                       
            else:
                while True:
                    try:
                        temp_bool = False
                        for m in self.keyword_dic[user_id]['main_text']:
                            if m != main_text:
                                temp_bool = False
                            else: 
                                temp_bool = True
                                break           
                        if temp_bool == False:
                            #print(1)
                            hashtags = [h.text[1:] for h in self.driver.find_elements_by_css_selector('ul.Xl2Pu>li>span>a')]
                            #print(2)
                            created_at = self.driver.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')
                            #print(3)
                            current_url = self.driver.current_url
                            #print(4)
                            # comments = [c.text for c in texts[1:]]
                            # likes = article.find_element_by_css_selector('span.zV_Nj>span').text
                    
                    except NoSuchElementException as ex:
                        print(i, 2, user_id, ex)
                    except Exception as e:
                        print(i, 2, user_id, e)
                    else:
                        self.keyword_dic[user_id]['main_text'].append(main_text)
                        self.keyword_dic[user_id]['hashtags'].append(hashtags)
                        self.keyword_dic[user_id]['created_at'].append(created_at)
                        self.keyword_dic[user_id]['current_url'].append(current_url)
                        self.keyword_dic[user_id]['comments'].append('/'.join(comments))
                        
                        likes = ''
                        
                        try:
                            likes = self.driver.find_element_by_css_selector('span.zV_Nj>span').text
                            self.keyword_dic[user_id]['likes'].append(likes)
                            self.dataset.append([user_id, created_at, main_text, hashtags, comments, likes, current_url])
                            break

                        except:
                            print('below 11')
                            self.keyword_dic[user_id]['likes'].append(likes)
                            self.dataset.append([user_id, created_at, main_text, hashtags, comments, likes, current_url])
                            break
                        
                        
                        # self.keyword_dic[user_id]['comments'].append(comments)
                        # self.keyword_dic[user_id]['likes'].append(likes)
                        
                    
            try:
                next_button = self.driver.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow')
                next_button.click()
                print(i, 'finished')
            except NoSuchElementException as ex:
                print('completed\n\n', ex)
            except Exception as e:
                print('completed\n\n', e)
            
                
    def save_data(self, data, datatype='list'):
        with open('{}.txt'.format(self.file_name + '_' + datatype), 'wb') as f:
            pickle.dump(data, f)
    '''
    def save_with_list(self):
        with open('{}_list.txt'.format(self.keyword), 'wb') as f:
            dataset = self.keyword.values()
            pickle.dump(datasets)
    '''
    def load_data(self, filename):
        with open(filename, 'rb') as f:
            dataset = pickle.load(f)
        return dataset
    
    def main(self):
        self.chrome_open()
        self.insta_keyword()
        self.crawling()
        self.save_data(self.keyword_dic, 'dic')
        self.save_data(self.dataset, 'list')
        self.driver.quit()

In [2]:
intake = Insta(keyword = '인테이크', number = '100', file_name = 'intake', mode='None')

In [3]:
intake.main()

get user_id


0 finished
1 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

get user_id


1 finished
2 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

get user_id


2 finished
get user_id


3 finished
4 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

get user_id


4 finished
get user_id


5 finished
6 user_id Message: stale element reference: element is not attached to the page document
  (Sessi

below 11
76 finished
get user_id


77 finished
get user_id


78 finished
get user_id


79 finished
get user_id


80 finished
get user_id


81 finished
get user_id


82 finished
83 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

get user_id


83 finished
84 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

get user_id


84 finished
get user_id


85 finished
get user_id


86 finished
get user_id


87 finished
get user_id


below 11
88 finished
get user_id


89 finished
90 user_id Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.99)


In [1]:
#intake.chrome_open()
intake.insta_keyword()

NameError: name 'intake' is not defined

In [28]:
intake.driver.quit()

In [6]:
def save_data(data, datatype='list'):
    with open('{}.txt'.format('intake_' + datatype), 'wb') as f:
        pickle.dump(data, f)
        
save_data(intake.dataset)
save_data(intake.keyword_dic, 'dic')
        


test = Insta(keyword = '모닝죽검은콩', number = 'total', dir_name = '모닝죽검은콩', mode='headless')

test.chrome_open()

test.insta_keyword()

time.sleep(3)

test.crawling()

test.save_data()

test.driver.quit()


print(len(list(test.keyword_dic.items())))


main_text = [len(test.keyword_dic[i]['main_text']) for i in test.keyword_dic]

print(sum(main_text))

import pickle
with open('test.txt', 'wb') as f:
    pickle.dump(test.keyword_dic, f)

with open('test.txt', 'rb') as f:
    test_dic = pickle.load(f)

test_dic['intakefoods']

test_list = list(test_dic.values())

test_list[:2]



test.keyword_dic['intakefoods']

print(test.user_ids)

In [60]:
a = [i for i in keyword_dic.keys()]

In [ ]:
a[100]

In [ ]:
    checker = True
    while checker:
        try:
            WebDriverWait(intake.driver, 3).until(EC.text_to_be_present_in_element_by_css_selector('ul.Xl2Pu>li>span>a:last-child', hashtags[-1]))
            print(1)
        except:
            checker = False
            print(2)
                                                  
                                                  
    # waiting = WebDriverWait(intake.driver, 3).until(EC.staleness_of(main_text.find_elements_by_tag_name('a')[-1]))
    
        

In [18]:
intake.driver.quit()

In [32]:
hashtags[0]

'.myday_1027'

In [ ]:
keyword_dic[a[100]]

In [60]:
texts = intake.driver.find_element_by_css_selector('ul.Xl2Pu > li:last-child').text

In [61]:
texts

'eo_s__g행님 광주에서머함'

In [168]:
a = [z.text for z in texts[0].find_elements_by_tag_name('a')]

In [170]:
a[1]

'#점심'

In [51]:
len(keyword_dic)

661

In [52]:
all_h = []
for i in keyword_dic:
    for j in keyword_dic[i]['hashtags']:
        all_h.extend(j)

In [ ]:
all_h

In [54]:
word_count = Counter(all_h)
word_count2 = []
for i in word_count:
    word_count2.append((i, word_count[i]))

word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)

In [ ]:
word_count2

In [45]:
user_id = article.find_element_by_css_selector('div.e1e1d').text

In [46]:
for i in 

'd_ddang2'

In [72]:
texts = article.find_elements_by_css_selector('ul.Xl2Pu > li')

In [73]:
main_text = texts[0]

In [ ]:
comments = texts[1:]

In [74]:
main_text.text

'd_ddang2#땅180617 #땅_아침🌿\n.\n밀스라이트(벨벳카카오)\n.\n오늘도 간단하게 챙겨나온 밀스라이트..! 근데 저번에 먹을때도 그랬지만 뚜껑 겁나 안열림...ㅠ 결국 칼로 뜯었었는데 오늘도 그랬다... 카페라떼 맛은 안 그런걸로 봐서 원래 그런건 아닌것 같고 불량품인가.. 남은것들도 다 칼로 따야하낰ㅋㅋㅋ 세상불편😭😭😭 아침에 바빠서 빨리먹어야 했는데 똑땅....다음병은 안그랬으면🙏\n.\n#아침식단#식단기록#다이어트식단#인테이크#밀스라이트#잇클린#다이어트#다이어터#다이어트그램'

In [75]:
hashtags = main_text.find_elements_by_tag_name('a')

In [76]:
for i in hashtags:
    print(i.text[1:])

_ddang2
땅180617
땅_아침🌿
아침식단
식단기록
다이어트식단
인테이크
밀스라이트
잇클린
다이어트
다이어터
다이어트그램


In [77]:
comments = texts[1:]

In [78]:
for i in comments:
    print(i.text)

neverminddarlingㅜㅠ 잉 땅이님 바쁘신데 요눔~~~~!! 나머지 병들은 똑 열리길 바랄게유
ddori_u아니진쨔..뚜껑..너의본분은..열리는것이란ㄴ다..잊으면안돼.. 땅이님 바쁜데 곤란하게하제말쟈..^_ㅜㅜㅜ
d_ddang2@neverminddarling 진짴ㅋㅋㅋㅋ 제가 힘이약한것도 아닌데 당황해써요😭
d_ddang2@ddori_u 앜ㅋㅋㅋㅋㅋ 또리님 졸귀...😆😆 후 뚜껑 부들부들... 밥먹게 해주라주!


In [79]:
likes = article.find_element_by_css_selector('span.zV_Nj>span').text

In [80]:
likes

'61'

In [81]:
next_button = intake.driver.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow')

In [82]:
next_button.click()

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException, NoSuchElementException, TimeoutException, StaleElementReferenceException, UnexpectedAlertPresentException
from selenium.webdriver.common.by import By
import os
import urllib.request
from collections import Counter


class Insta():

    def __init__(self, keyword, number, dir_name='images', mode='None'):
        self.keyword = keyword
        self.number = int(number)
        self.dir_name = dir_name
        self.mode = mode

    def chrome_open(self):

        self.options = webdriver.ChromeOptions()
        if self.mode == 'headless':
            self.options.add_argument('headless')
        self.options.add_argument('window-size=1920x1080')
        self.options.add_argument("disable-gpu")
        self.options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        self.options.add_argument("lang=ko_KR") # 한국어!
        self.options.add_argument("disable-infobars")
        self.driver = webdriver.Chrome('chromedriver', chrome_options=self.options)

        self.driver.get('http://google.com')
        self.driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")
        self.driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
        self.driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")

        self.driver.implicitly_wait(10)

    def crawling(self):
        keyword_t = urllib.parse.urlencode({'': self.keyword})[1:]
        self.driver.get('https://www.instagram.com/explore/tags/{}/?hl=ko'.format(keyword_t))
        #iframe = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'fb_xdm_frame_https')))
        #self.driver.switch_to.frame(iframe1)
        self.total = int(self.driver.find_element_by_xpath(
            '//*[@id="react-root"]/section/main/article/header/div[2]/span/span').text.replace(',', ''))

        self.txt_list = []
        self.img_url = []
        breaker = 0
        breaker = [0, 0]
        while True:
            imgs = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.eLAPa>div>img')))
            #imgs = WebDriverWait(self.driver, 10).until(
             #   EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div._e3il2>div>img')))
            for idx, i in enumerate(imgs):
                txt = i.get_attribute('alt')
                url = i.get_attribute('src')
                if not url in self.img_url:
                    num = len(self.img_url) + 1
                    self.img_url.append(url)
                    self.txt_list.append(txt)
                    urllib.request.urlretrieve(url, '{}/{}_{}.jpg'.format(self.dir_name, self.keyword, num))
                if num >= self.number:
                    break
                elif num >= self.total:
                    break

            breaker.append(len(self.img_url))
            try:
                if breaker[-1] == breaker[-10]:
                    break
            except Exception as e:
                pass

            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    def makedir(self, dir_name):
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

    def reviews(self):
        f = open('{}/{}_reviews.txt'.format(self.dir_name, self.keyword), 'w', encoding='utf-8')
        for idx, row in enumerate(self.txt_list):
            f.write(str(idx+1) + '.\n')
            f.write(row)
            f.write('\n\n\n')
        f.close()

        self.txt_merge = []
        for idx, row in enumerate(self.txt_list):
            self.txt_merge.extend(row.split('#'))
        for idx, row in enumerate(self.txt_merge):
            self.txt_merge[idx] = row.strip()

    def hashtags(self):
        word_count = Counter(self.txt_merge)

        word_count2 = []
        for i in word_count:
            word_count2.append((i, word_count[i]))

        word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)

        with open('{}/{}_count.txt'.format(self.dir_name, self.keyword), 'w', encoding='utf-8') as f:
            for i in word_count2:
                f.write(str(i) + '\n')

    def img_crawling(self):
        self.chrome_open()
        self.makedir(self.dir_name)
        self.crawling()

    def quit(self):
        self.driver.quit()




In [14]:
pajamas.reviews()

pajamas.hashtags()

pajamas.quit()

In [16]:
vr카페 = Insta(keyword = 'vr카페', number = 500, dir_name = 'vr카페', mode=None)

vr카페.img_crawling()

vr카페.reviews()

vr카페.hashtags()

vr카페.quit()